### Formatting Results for Competition

In [1]:
import itertools
import pandas as pd
from data_gathering import gather_data_features, gather_data_actuals

In [10]:

contribution_results_2018_val_f_m = pd.read_parquet(
    "mlruns/3/9d8897f7efd242bdb33fdfe0c8c1eb6a/artifacts/contribution_becker_drauz_BaselineModel_test_window_2018.parquet")
contribution_results_2018_val_f = pd.read_parquet(
    "mlruns/3/5b3acdf73aeb4eecb8bb007157f3535d/artifacts/contribution_becker_drauz_BaselineModel_test_window_2018.parquet")
contribution_results_2019_val_f_m = pd.read_parquet(
    "mlruns/3/80caad1b10bf4513b5b5fbb9e8827d49/artifacts/contribution_becker_drauz_BaselineModel_test_window_2019.parquet")
contribution_results_2019_val_f = pd.read_parquet(
    "mlruns/3/df6e7240234c40cebc96397963780cf9/artifacts/contribution_becker_drauz_BaselineModel_test_window_2019.parquet")
contribution_results_2020_val_f_m = pd.read_parquet(
    "mlruns/3/47685285ae8b45d38e966b28d5c27579/artifacts/contribution_becker_drauz_BaselineModel_test_window_2020.parquet")
contribution_results_2020_val_f = pd.read_parquet(
    "mlruns/3/301944cf5760482da0291a8e9daca4e1/artifacts/contribution_becker_drauz_BaselineModel_test_window_2020.parquet")
contribution_results_2021_val_f_m = pd.read_parquet(
    "mlruns/3/021e0fc647be407d8924e4d134093adc/artifacts/contribution_becker_drauz_BaselineModel_test_window_2021.parquet")
contribution_results_2021_val_f = pd.read_parquet(
    "mlruns/3/3cc10d101573487aa558f41b21de99c5/artifacts/contribution_becker_drauz_BaselineModel_test_window_2021.parquet")

In [5]:
# First, check each object's type
print("Data type checks:")
print("2018:", type(contribution_results_2018_val_f_m))
print("2019:", type(contribution_results_2019_val_f_m))
print("2020:", type(contribution_results_2020_val_f_m))
print("2021:", type(contribution_results_2021_val_f_m))


Data type checks:
2018: <class 'pandas.core.frame.DataFrame'>
2019: <class 'pandas.core.frame.DataFrame'>
2020: <class 'pandas.core.frame.DataFrame'>
2021: <class 'pandas.core.frame.DataFrame'>


In [11]:
# Concat the contribution results of four years into one dataframe
contribution_results_allyears_val_f_m = pd.concat(
    [contribution_results_2018_val_f_m, 
     contribution_results_2019_val_f_m, 
     contribution_results_2020_val_f_m,
     contribution_results_2021_val_f_m]).reset_index(drop=True)
contribution_results_allyears_val_f = pd.concat(
    [contribution_results_2018_val_f, 
    contribution_results_2019_val_f,
    contribution_results_2020_val_f, 
    contribution_results_2021_val_f]).reset_index(drop=True)

In [12]:
# Remove column draw from the dataframes
contribution_results_allyears_val_f_m = contribution_results_allyears_val_f_m.drop(columns=['draw'])
contribution_results_allyears_val_f = contribution_results_allyears_val_f.drop(columns=['draw'])

# Rename each column that starts with "sample_" to "draw_"
contribution_results_allyears_val_f_m = contribution_results_allyears_val_f_m.rename(
    columns={col: col.replace('sample', 'draw') for col in contribution_results_2018_val_f_m.columns})
contribution_results_allyears_val_f = contribution_results_allyears_val_f.rename(
    columns={col: col.replace('sample', 'draw') for col in contribution_results_2018_val_f.columns})

# Export the dataframes to parquet files in export folder
export_path = "C:/Users/Uwe Drauz/Documents/bachelor_thesis_local/personal_competition_data/Results"
contribution_results_allyears_val_f_m.to_parquet(fr"{export_path}/baseline_f_m_posterior_predictive_samples.parquet")
contribution_results_allyears_val_f.to_parquet(fr"{export_path}/baseline_f_posterior_predictive_samples.parquet")

In [4]:
# Create a nested dictionary with key1: "validation_on_forecast_horizon" (_f in df) and key2: "validation_on_forecast_horizon_and_month" (_f_m in df). The inner dictionaires contain the dataframes with the contribution results for each year
contribution_results = {
    "validate_on_forecast_horizon": {
        2018: contribution_results_2018_val_f,
        2019: contribution_results_2019_val_f,
        2020: contribution_results_2020_val_f,
        2021: contribution_results_2021_val_f
    },
    "validate_on_forecast_horizon_and_month": {
        2018: contribution_results_2018_val_f_m,
        2019: contribution_results_2019_val_f_m,
        2020: contribution_results_2020_val_f_m,
        2021: contribution_results_2021_val_f_m
    }
}
test_window_dict = {
    "validate_on_forecast_horizon": {
        2018: None,
        2019: None,
        2020: None,
        2021: None
    },
    "validate_on_forecast_horizon_and_month": {
        2018: None,
        2019: None,
        2020: None,
        2021: None
    }
}

In [7]:
# Load 'Actuals' data
data_cm_actual_2018, data_cm_actual_2019, data_cm_actual_2020, data_cm_actual_2021, data_cm_actual_allyears \
    = gather_data_actuals()
# Load features data
data_cm_features_2017, data_cm_features_2018, data_cm_features_2019, data_cm_features_2020, data_cm_features_allyears \
    = gather_data_features()

In [8]:
# Determine all countries
actual_countries = data_cm_actual_allyears['country_id'].unique()
# Determine countries with at least one conflict fatality
feature_countries_non_zero = data_cm_features_allyears[data_cm_features_allyears['ged_sb'] > 0][
    'country_id'].unique()
# Determine countries which have at least one conflict fatality and are in the actuals data
feature_and_actuals_countries_non_zero = list(set(feature_countries_non_zero) & set(actual_countries))

# Determine the counties which are in actual_countries but not in feature_and_actuals_countries_non_zero
countries_in_actuals_without_observations = list(
    set(actual_countries) - set(feature_and_actuals_countries_non_zero))

In [7]:
countries_in_results = contribution_results_2021_val_f['country_id'].unique()

In [5]:
def prepare_dataframe_for_formatting(df: pd.DataFrame, missing_country_ids: list):
    if 'draw' in df.columns:
        df = df.drop(columns=['draw'])
    # Unique month_ids in the original DataFrame
    unique_month_ids = df['month_id'].unique()
    # Create an empty list to store new rows
    new_rows = []
    
    # Generate new rows
    for month_id in unique_month_ids:
        for country_id in missing_country_ids:
            new_row = {'month_id': month_id, 'country_id': country_id}
            for sample_col in [f'sample_{i}' for i in range(1, 100)]:
                new_row[sample_col] = 0.0
            new_rows.append(new_row)
    
    # Create a new DataFrame from the list of new rows
    new_df = pd.DataFrame(new_rows)
    
    # Append the new DataFrame to the original DataFrame
    df = pd.concat([df, new_df], ignore_index=True)
    
    # Optionally, you can sort the DataFrame
    df = df.sort_values(by=['country_id', 'month_id']).reset_index(drop=True)
    
    return df

    

In [14]:
# Prepare each datafame in the contribution_results dictionary for formatting
for validation_type in contribution_results.keys():
    for year in contribution_results[validation_type].keys():
        df = contribution_results[validation_type][year]
        df = prepare_dataframe_for_formatting(df, countries_in_actuals_without_observations)
        contribution_results[validation_type][year] = df

In [6]:
# OPTIONAL: Implementation if not all country, month combinations have the same numer of samples, could be like this:
        # # Check if there are nan values in the sample_values list and drop them if there are
        # if any([pd.isna(sample_value) for sample_value in sample_values]):
        #     sample_values = [sample_value for sample_value in sample_values if not pd.isna(sample_value)]
def format_dataframe_for_competition(df: pd.DataFrame):
    results_formatted = pd.DataFrame()
    month_ids = df['month_id'].unique()
    country_ids = df['country_id'].unique()
    for month_id, country_id in itertools.product(month_ids, country_ids):
        filtered_df = df[(df['month_id'] == month_id) & (df['country_id'] == country_id)]
        sample_columns = [col for col in df.columns if 'sample_' in col]
        sample_values = filtered_df[sample_columns].values.tolist()
        if len(sample_values) > 0:
            sample_values = sample_values[0]
        draw = list(range(0, len(sample_values)))
        index_tuple_list = []
        for i in range(len(sample_values)):
            index_tuple_list.append((month_id, country_id, draw[i]))
        index = pd.MultiIndex.from_tuples(index_tuple_list, names=['month_id', 'country_id', 'draw'])
        outcome = sample_values
        df_single_iteration = pd.DataFrame({"outcome": outcome}, index=index)
        results_formatted = pd.concat([results_formatted, df_single_iteration], ignore_index=False)
        
    return results_formatted 

In [16]:
# Format each datafame in the contribution_results dictionary and store it in the test_window_dict
for validation_type in contribution_results.keys():
    for year in contribution_results[validation_type].keys():
        df = contribution_results[validation_type][year]
        df = format_dataframe_for_competition(df)
        test_window_dict[validation_type][year] = df

In [20]:
# Define path for export
contribution_export_path = "competition_results"
# Export the formatted dataframes to parquet files as f"test_window_{year}_baseline_{validation_type}.parquet"
for validation_type in test_window_dict.keys():
    for year in test_window_dict[validation_type].keys():
        df = test_window_dict[validation_type][year]
        df.to_parquet(fr"{contribution_export_path}/test_window_{year}_baseline_{validation_type}.parquet")

In [4]:
dict_actuals = {
        "2018": data_cm_actual_2018,
        "2019": data_cm_actual_2019,
        "2020": data_cm_actual_2020,
        "2021": data_cm_actual_2021
        }

In [24]:
# Compare for every dataframe in the test_window_dict if the unique countries and months are the same unique countries and months as in the actuals data
for validation_type in test_window_dict.keys():
    for year in test_window_dict[validation_type].keys():
        df = test_window_dict[validation_type][year]
        unique_countries = df.index.get_level_values('country_id').unique()
        unique_months = df.index.get_level_values('month_id').unique()
        actual_countries = dict_actuals[str(year)]['country_id'].unique()
        actual_months = dict_actuals[str(year)]['month_id'].unique()
        if not set(unique_countries) == set(actual_countries):
            print(f"Unique countries in {validation_type} {year} are not the same as in the actuals data")
        else: 
            print(f"Unique countries in {validation_type} {year} are the same as in the actuals data")
        if not set(unique_months) == set(actual_months):
            print(f"Unique months in {validation_type} {year} are not the same as in the actuals data")
        else:
            print(f"Unique months in {validation_type} {year} are the same as in the actuals data")

Unique countries in validate_on_forecast_horizon 2018 are the same as in the actuals data
Unique months in validate_on_forecast_horizon 2018 are the same as in the actuals data
Unique countries in validate_on_forecast_horizon 2019 are the same as in the actuals data
Unique months in validate_on_forecast_horizon 2019 are the same as in the actuals data
Unique countries in validate_on_forecast_horizon 2020 are the same as in the actuals data
Unique months in validate_on_forecast_horizon 2020 are the same as in the actuals data
Unique countries in validate_on_forecast_horizon 2021 are the same as in the actuals data
Unique months in validate_on_forecast_horizon 2021 are the same as in the actuals data
Unique countries in validate_on_forecast_horizon_and_month 2018 are the same as in the actuals data
Unique months in validate_on_forecast_horizon_and_month 2018 are the same as in the actuals data
Unique countries in validate_on_forecast_horizon_and_month 2019 are the same as in the actuals 

In [25]:
# Print the no of rows of each dataframe in the test_window_dict
for validation_type in test_window_dict.keys():
    for year in test_window_dict[validation_type].keys():
        df = test_window_dict[validation_type][year]
        print(f"Number of rows in {validation_type} {year}: {df.shape[0]}")

Number of rows in validate_on_forecast_horizon 2018: 226908
Number of rows in validate_on_forecast_horizon 2019: 226908
Number of rows in validate_on_forecast_horizon 2020: 226908
Number of rows in validate_on_forecast_horizon 2021: 226908
Number of rows in validate_on_forecast_horizon_and_month 2018: 226908
Number of rows in validate_on_forecast_horizon_and_month 2019: 226908
Number of rows in validate_on_forecast_horizon_and_month 2020: 226908
Number of rows in validate_on_forecast_horizon_and_month 2021: 226908


In [26]:
print(12 * 191 * 99)

226908


Test reformatting for Bayesian Model

In [ ]:
bayesian_results = pd.read_parquet(
    r'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\Results\model3_zinb_feature_set1_posterior_predicitve_samples.parquet')
# Change the word "draw" to "sample" in the column names of the bayesian_results dataframe
bayesian_results.columns = [col.replace('draw', 'sample') for col in bayesian_results.columns]
# Drop "ged_sb" column
bayesian_results = bayesian_results.drop(columns=['ged_sb'])

In [9]:
# Prepare the bayesian_results dataframe for formatting
bayesian_results = prepare_dataframe_for_formatting(bayesian_results, countries_in_actuals_without_observations)

In [10]:
# Format the bayesian_results dataframe
bayesian_results_formatted = format_dataframe_for_competition(bayesian_results)

In [11]:
# Format one of the baseline dataframes for comparison
baseline_results = prepare_dataframe_for_formatting(contribution_results_2021_val_f, countries_in_actuals_without_observations)
baseline_results_formatted = format_dataframe_for_competition(baseline_results)